## 3.0 Импорты библиотек

In [1]:
import os
import yaml
import logging
import numpy as np
import scipy.stats as stats
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Any, Optional, Tuple, Dict
from datetime import datetime
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from scipy.stats import pearsonr, spearmanr, kurtosis, skew
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# расширяем поле ноутбука для удобства
from IPython.display import display, HTML
display(HTML('<style>.container {width:87% !important;}</style>'))
display(HTML("<style>.output_scroll {height:auto !important; max-height:10000px !important;}</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
# Настройки для pandas (количество отображаемых колонок)
pd.set_option('display.max_columns', 100)

In [5]:
# Определение стиля для pyplot
plt.style.use('ggplot')

In [6]:
# Текущая рабочая директория
cwd = Path().resolve()

# Поднимаемся на один уровень выше
project_root = cwd.parent

# Добавляем корень проекта в sys.path
sys.path.append(str(project_root))

# Загрузка данных из config.yaml
from src.data import downloader, loader, preprocessor, saving

# Путь к файлу config.yaml
config_path = project_root / "config" / "config.yaml"

# Загружаем конфиг
config = loader.load_config(config_path)

## 3.1. Загрузка данных

In [7]:
# Загрузка train
df_train = loader.data_load_preprocessed(data_type='train', config=config)

[⧗] Загружаю данные из: ..\data/processed\eda_data_train.pkl
[✓] Данные успешно загружены. Форма: (781, 11)


In [8]:
# Вывод первых 5 строк тренировочного датасета
df_train.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength,W/C,Sp/C_pct
0,376.0,0.0,0.0,214.6,0.0,1003.5,762.4,3,16.28,0.570745,0.000000
1,491.0,26.0,123.0,210.0,3.9,882.0,699.0,56,59.59,0.427699,0.007943
2,250.0,0.0,95.7,187.4,5.5,956.9,861.2,3,13.82,0.749600,0.022000
3,310.0,0.0,0.0,192.0,0.0,1012.0,830.0,90,35.76,0.619355,0.000000
4,252.1,97.1,75.6,193.8,8.3,835.5,821.4,28,33.40,0.768743,0.032923


In [9]:
# Загрузка test
df_test = loader.data_load_preprocessed(data_type='test', config=config)

[⧗] Загружаю данные из: ..\data/processed\eda_data_test.pkl
[✓] Данные успешно загружены. Форма: (228, 10)


In [10]:
# Вывод первых 5 строк тестового датасета
df_test.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,W/C,Sp/C_pct
0,167.4,129.9,128.6,175.5,7.8,1006.3,746.6,28,1.048387,0.046595
1,475.0,118.8,0.0,181.1,8.9,852.1,781.5,7,0.381263,0.018737
2,251.4,0.0,118.3,188.5,6.4,1028.4,757.7,100,0.749801,0.025457
3,307.0,0.0,0.0,193.0,0.0,968.0,812.0,365,0.628664,0.000000
4,143.6,0.0,174.9,158.4,17.9,942.7,844.5,28,1.103064,0.124652


## 3.2. Обработка выбросов

### Объявление класса

In [11]:
class OutlierHandler(BaseEstimator, TransformerMixin):
    """
    Обработчик выбросов для использования в sklearn Pipeline.
    Поддерживает различные стратегии обработки выбросов.
    """
    
    def __init__(self, 
                 strategies: List[str] = ['abnormal'],
                 config: Optional[Dict] = None,
                 target_col: str = None):
        """
        Инициализация обработчика выбросов.
        
        Args:
            strategies: Список стратегий обработки 
                      ['abnormal', 'combine', 'gost_binar', 'gost_remove', 'iqr_remove']
            config: Словарь с конфигурацией ГОСТ диапазонов и аномальных значений
            target_col: Название целевой переменной (например, 'Strength')
        """
        self.strategies = strategies
        self.config = config or self._get_default_config()
        self.target_col = target_col
        self.iqr_bounds_ = {}
        self.feature_names_in_ = None
        self.fitted_ = False
        self.removed_indices_ = set()
        self.outlier_summary_ = {}
        
        # Инициализация порогов из конфигурации
        self._init_thresholds()
    
    def _get_default_config(self):
        """Конфигурация по умолчанию если не предоставлена"""
        return {
            'standard_value': {
                'age': {'max': 365, 'min': 1},
                'cement': {'max': 600, 'min': 200},
                'coarse_aggregate': {'max': 1300, 'min': 1000},
                'fine_aggregate': {'max': 800, 'min': 600},
                'fly_ash': {'max': 200, 'min': 0},
                'sp_c_pct': {'max': 0.025, 'min': 0.005},
                'strength': {'max': 100, 'min': 5},
                'w_c': {'max': 0.7, 'min': 0.3},
                'water': {'max': 220, 'min': 120}
            },
            'abnormal_value': {
                'sp_c_pct': 0.07,
                'fine_aggregate': 970
            }
        }
    
    def _init_thresholds(self):
        """Инициализация порогов из конфигурации"""
        # ГОСТ диапазоны
        self.gost_ranges = self.config.get('standard_value', {})
        
        # Аномальные значения
        abnormal_config = self.config.get('abnormal_value', {})
        self.abnormal_thresholds = {
            'sp_c_pct': abnormal_config.get('sp_c_pct', 0.07),
            'fine_aggregate': abnormal_config.get('fine_aggregate', 970)
        }
        
        # Бинарные пороги для combine стратегии
        self.combine_thresholds = {
            'sp_critical': self.abnormal_thresholds['sp_c_pct'],
            'sp_warning': self.gost_ranges.get('sp_c_pct', {}).get('max', 0.025),
            'fa_critical': self.abnormal_thresholds['fine_aggregate'],
            'fa_warning': self.gost_ranges.get('fine_aggregate', {}).get('max', 800)
        }
    
    def fit(self, X, y=None):
        """
        Обучение обработчика на данных.
        
        Args:
            X: DataFrame с признаками
            y: Целевая переменная (опционально)
            
        Returns:
            self: Обученный трансформер
        """
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
            
        self.feature_names_in_ = X.columns.tolist()
        
        # Сохранение индексов для синхронизации с y
        self.original_indices_ = X.index
        
        # Расчет IQR границ если нужна стратегия iqr_remove
        if 'iqr_remove' in self.strategies:
            self._calculate_iqr_bounds(X)
        
        # Инициализация сводки
        self._init_outlier_summary(X)
        
        self.fitted_ = True
        return self
    
    def _calculate_iqr_bounds(self, X):
        """Расчет IQR границ для числовых признаков"""
        numeric_features = X.select_dtypes(include=[np.number]).columns
        
        for feature in numeric_features:
            Q1 = X[feature].quantile(0.25)
            Q3 = X[feature].quantile(0.75)
            IQR = Q3 - Q1
            
            self.iqr_bounds_[feature] = {
                'lower': Q1 - 1.5 * IQR,
                'upper': Q3 + 1.5 * IQR,
                'q1': Q1,
                'q3': Q3
            }
    
    def _init_outlier_summary(self, X):
        """Инициализация сводки по выбросам"""
        self.outlier_summary_ = {
            'original_shape': X.shape,
            'strategies_applied': [],
            'removed_rows': 0,
            'binary_features_created': [],
            'outliers_by_feature': {},
            'removed_indices': set()
        }
    
    def transform(self, X, y=None):
        """
        Применение обработки выбросов к данным.
        
        Args:
            X: DataFrame с признаками
            y: Целевая переменная (опционально)
            
        Returns:
            X_processed: Обработанный DataFrame
            y_processed: Обработанная целевая переменная (если y предоставлена)
        """
        if not self.fitted_:
            raise ValueError("Сначала необходимо вызвать fit()")
            
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
            
        X_processed = X.copy()
        y_processed = y.copy() if y is not None else None
        
        # Сброс сводки
        self.removed_indices_ = set()
        self._init_outlier_summary(X)
        
        # Применение стратегий в указанном порядке
        for strategy in self.strategies:
            if strategy == 'abnormal':
                X_processed, y_processed = self._apply_abnormal_strategy(X_processed, y_processed)
            elif strategy == 'combine':
                X_processed, y_processed = self._apply_combine_strategy(X_processed, y_processed)
            elif strategy == 'gost_binar':
                X_processed = self._apply_gost_binar_strategy(X_processed)
            elif strategy == 'gost_remove':
                X_processed, y_processed = self._apply_gost_remove_strategy(X_processed, y_processed)
            elif strategy == 'iqr_remove':
                X_processed, y_processed = self._apply_iqr_remove_strategy(X_processed, y_processed)
            else:
                print(f"Неизвестная стратегия: {strategy}")
        
        # Обновление сводки
        self.outlier_summary_['final_shape'] = X_processed.shape
        self.outlier_summary_['removed_rows'] = len(self.removed_indices_)
        self.outlier_summary_['removed_indices'] = self.removed_indices_
        
        if y is not None:
            return X_processed, y_processed
        return X_processed
    
    def _apply_abnormal_strategy(self, X, y=None):
        """Стратегия abnormal: удаление критических выбросов"""
        rows_to_remove = set()
        
        # Superplasticizer >= порог удалить строки
        sp_col = 'Sp/C_pct' if 'Sp/C_pct' in X.columns else 'sp_c_pct'
        if sp_col in X.columns:
            sp_threshold = self.abnormal_thresholds['sp_c_pct']
            sp_outliers = X[X[sp_col] >= sp_threshold].index
            rows_to_remove.update(sp_outliers)
            self._update_outlier_summary('Sp/C_pct', len(sp_outliers), sp_threshold)
        
        # Fine Aggregate >= порог удалить строки
        fa_col = 'Fine Aggregate' if 'Fine Aggregate' in X.columns else 'fine_aggregate'
        if fa_col in X.columns:
            fa_threshold = self.abnormal_thresholds['fine_aggregate']
            fa_outliers = X[X[fa_col] > fa_threshold].index
            rows_to_remove.update(fa_outliers)
            self._update_outlier_summary('Fine Aggregate', len(fa_outliers), fa_threshold)
        
        return self._remove_rows(X, y, rows_to_remove, 'abnormal')
    
    def _apply_combine_strategy(self, X, y=None):
        """Стратегия combine: комбинированная обработка"""
        rows_to_remove = set()
        
        # Обработка Superplasticizer
        sp_col = 'Sp/C_pct' if 'Sp/C_pct' in X.columns else 'sp_c_pct'
        if sp_col in X.columns:
            # Удаление строк с SP >= критический порог
            sp_critical = self.combine_thresholds['sp_critical']
            sp_remove = X[X[sp_col] >= sp_critical].index
            rows_to_remove.update(sp_remove)
            
            # Бинарный признак для SP >= предупреждающий порог
            sp_warning = self.combine_thresholds['sp_warning']
            X['High_SP'] = (X[sp_col] >= sp_warning).astype(int)
            high_sp_count = X['High_SP'].sum()
            
            self.outlier_summary_['binary_features_created'].append('High_SP')
            self._update_outlier_summary('High_SP', high_sp_count, sp_warning)
        
        # Обработка Fine Aggregate
        fa_col = 'Fine Aggregate' if 'Fine Aggregate' in X.columns else 'fine_aggregate'
        if fa_col in X.columns:
            # Удаление строк с FA >= критический порог
            fa_critical = self.combine_thresholds['fa_critical']
            fa_remove = X[X[fa_col] >= fa_critical].index
            rows_to_remove.update(fa_remove)
            
            # Бинарный признак для FA >= предупреждающий порог
            fa_warning = self.combine_thresholds['fa_warning']
            X['High_FA'] = (X[fa_col] >= fa_warning).astype(int)
            high_fa_count = X['High_FA'].sum()
            
            self.outlier_summary_['binary_features_created'].append('High_FA')
            self._update_outlier_summary('High_FA', high_fa_count, fa_warning)
        
        return self._remove_rows(X, y, rows_to_remove, 'combine')
    
    def _apply_gost_binar_strategy(self, X):
        """Стратегия gost_binar: бинарные признаки по ГОСТ"""
        if not self.gost_ranges:
            print("GOST binar: Конфигурация ГОСТ не найдена")
            return X
        
        for feature, ranges in self.gost_ranges.items():
            # Поиск соответствующего столбца в данных
            data_col = self._find_matching_column(X, feature)
            if data_col:
                min_val = ranges.get('min')
                max_val = ranges.get('max')
                
                if min_val is not None and max_val is not None:
                    # Бинарный признак для значений вне ГОСТ диапазона
                    outlier_mask = (X[data_col] < min_val) | (X[data_col] > max_val)
                    binary_feature_name = f'Outlier_{feature}'
                    X[binary_feature_name] = outlier_mask.astype(int)
                    
                    outlier_count = outlier_mask.sum()
                    if outlier_count > 0:
                        self.outlier_summary_['binary_features_created'].append(binary_feature_name)
                        self._update_outlier_summary(feature, outlier_count, f"{min_val}-{max_val}")
        
        return X
    
    def _apply_gost_remove_strategy(self, X, y=None):
        """Стратегия gost_remove: удаление по ГОСТ"""
        if not self.gost_ranges:
            print("GOST remove: Конфигурация ГОСТ не найдена")
            return X, y
            
        rows_to_remove = set()
        
        for feature, ranges in self.gost_ranges.items():
            # Поиск соответствующего столбца в данных
            data_col = self._find_matching_column(X, feature)
            if data_col:
                min_val = ranges.get('min')
                max_val = ranges.get('max')
                
                if min_val is not None and max_val is not None:
                    outliers = X[(X[data_col] < min_val) | (X[data_col] > max_val)].index
                    rows_to_remove.update(outliers)
                    
                    if outliers.any():
                        self._update_outlier_summary(feature, len(outliers), f"{min_val}-{max_val}")
        
        return self._remove_rows(X, y, rows_to_remove, 'gost_remove')
    
    def _apply_iqr_remove_strategy(self, X, y=None):
        """Стратегия iqr_remove: удаление по IQR"""
        if not self.iqr_bounds_:
            print("IQR remove: IQR границы не рассчитаны")
            return X, y
            
        rows_to_remove = set()
        
        for feature, bounds in self.iqr_bounds_.items():
            if feature in X.columns:
                outliers = X[
                    (X[feature] < bounds['lower']) | 
                    (X[feature] > bounds['upper'])
                ].index
                rows_to_remove.update(outliers)
                
                if outliers.any():
                    self._update_outlier_summary(
                        f"IQR_{feature}", 
                        len(outliers), 
                        f"{bounds['lower']:.2f}-{bounds['upper']:.2f}"
                    )
        
        return self._remove_rows(X, y, rows_to_remove, 'iqr_remove')
    
    def _find_matching_column(self, X, feature_name):
        """Поиск соответствующего столбца в данных"""
        # Прямое соответствие
        if feature_name in X.columns:
            return feature_name
        
        # Поиск похожих названий
        possible_matches = {
            'cement': ['Cement', 'cement'],
            'water': ['Water', 'water'],
            'fine_aggregate': ['Fine Aggregate', 'FineAggregate', 'fine_aggregate'],
            'coarse_aggregate': ['Coarse Aggregate', 'CoarseAggregate', 'coarse_aggregate'],
            'fly_ash': ['Fly Ash', 'FlyAsh', 'fly_ash'],
            'sp_c_pct': ['Sp/C_pct', 'Superplasticizer_pct', 'sp_c_pct'],
            'w_c': ['W/C', 'Water_Cement_Ratio', 'w_c'],
            'age': ['Age', 'age'],
            'strength': ['Strength', 'strength']
        }
        
        if feature_name in possible_matches:
            for match in possible_matches[feature_name]:
                if match in X.columns:
                    return match
        
        return None
    
    def _remove_rows(self, X, y, rows_to_remove, strategy_name):
        """Удаление строк из X и y"""
        if rows_to_remove:
            self.removed_indices_.update(rows_to_remove)
            self.outlier_summary_['strategies_applied'].append(
                f"{strategy_name}: {len(rows_to_remove)} rows"
            )
            
            X_processed = X.drop(index=rows_to_remove)
            y_processed = y.drop(index=rows_to_remove) if y is not None else None
            
            return X_processed, y_processed
        
        return X, y
    
    def _update_outlier_summary(self, feature, count, threshold):
        """Обновление сводки по выбросам"""
        if feature not in self.outlier_summary_['outliers_by_feature']:
            self.outlier_summary_['outliers_by_feature'][feature] = []
        
        self.outlier_summary_['outliers_by_feature'][feature].append({
            'count': count,
            'threshold': threshold
        })
    
    def fit_transform(self, X, y=None):
        """Обучение и преобразование в одном методе"""
        return self.fit(X, y).transform(X, y)
    
    def get_feature_names_out(self, input_features=None):
        """Получение имен признаков после преобразования"""
        if not self.fitted_:
            raise ValueError("Сначала необходимо вызвать fit()")
        return self.feature_names_in_
    
    def get_outlier_summary(self):
        """Получение сводки по обработке выбросов"""
        return self.outlier_summary_
    
    def visualize_outliers(self, X, features=None, figsize=(15, 10)):
        """
        Визуализация выбросов до и после обработки.
        
        Args:
            X: Исходные данные
            features: Список признаков для визуализации
            figsize: Размер фигуры
        """
        if not self.fitted_:
            raise ValueError("Сначала необходимо вызвать fit()")
        
        if features is None:
            # Выбор числовых признаков для визуализации
            numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
            features = numeric_features[:6]  # Ограничим 6 признаками
        
        n_features = len(features)
        n_cols = 2
        n_rows = (n_features + 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
        axes = axes.flatten() if n_features > 1 else [axes]
        
        for i, feature in enumerate(features):
            if i >= len(axes):
                break
                
            ax = axes[i]
            
            # Данные до обработки
            data_before = X[feature]
            
            # Данные после обработки
            X_processed = self.transform(X)
            data_after = X_processed[feature] if feature in X_processed.columns else pd.Series([])
            
            # Построение boxplot
            data_to_plot = [data_before.dropna()]
            labels = ['Before']
            
            if len(data_after) > 0:
                data_to_plot.append(data_after.dropna())
                labels.append('After')
            
            ax.boxplot(data_to_plot, labels=labels)
            ax.set_title(f'Outliers: {feature}')
            ax.set_ylabel('Values')
            
            # Добавление информации о выбросах
            outliers_before = self._count_outliers_iqr(data_before)
            outliers_after = self._count_outliers_iqr(data_after) if len(data_after) > 0 else 0
            
            ax.text(0.5, 0.95, f'Outliers: {outliers_before}', 
                   transform=ax.transAxes, ha='center', va='top',
                   bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue"))
            
            if len(data_after) > 0:
                ax.text(1.5, 0.95, f'Outliers: {outliers_after}', 
                       transform=ax.transAxes, ha='center', va='top',
                       bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen"))
        
        # Скрываем пустые subplots
        for j in range(i + 1, len(axes)):
            axes[j].set_visible(False)
        
        plt.tight_layout()
        plt.show()
        
        # Вывод сводки
        self._print_processing_summary()
    
    def _count_outliers_iqr(self, data):
        """Подсчет выбросов по IQR"""
        if len(data) == 0:
            return 0
        
        Q1 = data.quantile(0.25)
        Q3 = data.quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        return ((data < lower_bound) | (data > upper_bound)).sum()
    
    def _print_processing_summary(self):
        """Вывод сводки по обработке"""
        summary = self.get_outlier_summary()
        
        print("=" * 50)
        print("OUTLIER PROCESSING SUMMARY")
        print("=" * 50)
        print(f"Original data shape: {summary['original_shape']}")
        print(f"Final data shape: {summary.get('final_shape', 'N/A')}")
        print(f"Rows removed: {summary['removed_rows']}")
        print(f"Applied strategies: {', '.join(summary['strategies_applied'])}")
        
        if summary['binary_features_created']:
            print(f"Binary features created: {', '.join(summary['binary_features_created'])}")
        
        print("\nOutliers by feature:")
        for feature, outliers in summary['outliers_by_feature'].items():
            for outlier_info in outliers:
                print(f"  - {feature}: {outlier_info['count']} (threshold: {outlier_info['threshold']})")
        
        print("=" * 50)

### Проверка

In [12]:
# Разделение на признаки и целевую переменную
X = df_train.drop('Strength', axis=1)
y = df_train['Strength']
y_name = y.name

In [13]:
# Все доступные стратегии
all_strategies = [
    ['abnormal'],
    ['combine'],
    ['gost_binar'],
    ['gost_remove'],
    ['iqr_remove'],
    ['gost_binar', 'combine'],
]

In [14]:
print("=" * 70)
print("СРАВНЕНИЕ СТРАТЕГИЙ ОБРАБОТКИ ВЫБРОСОВ")
print("=" * 70)
print(f"Исходные данные: {X.shape} строк, {X.shape[1]} признаков")
print()

results = []

СРАВНЕНИЕ СТРАТЕГИЙ ОБРАБОТКИ ВЫБРОСОВ
Исходные данные: (781, 10) строк, 10 признаков



In [15]:
# Цикл по всем стратегиям
for i, strategies in enumerate(all_strategies, 1):
    print(f"\n{'='*50}")
    print(f"СТРАТЕГИЯ {i}: {strategies}")
    print(f"{'='*50}")
    
    # Создание обработчика для текущей стратегии
    outlier_handler = OutlierHandler(
        strategies=strategies,
        config=config,
        target_col=y_name
    )
    
    try:
        # Обработка данных
        X_processed, y_processed = outlier_handler.fit_transform(X, y)
        
        # Получение сводки
        summary = outlier_handler.get_outlier_summary()
        
        # Сохранение результатов
        result = {
            'strategies': strategies,
            'X_original': X.shape[0],
            'X_processed': X_processed.shape[0],
            'rows_removed': summary['removed_rows'],
            'removal_percent': (summary['removed_rows'] / X.shape[0]) * 100,
            'binary_features': len(summary['binary_features_created']),
            'outlier_handler': outlier_handler,
            'X_processed_data': X_processed,
            'y_processed_data': y_processed
        }
        results.append(result)
        
        # Вывод результатов
        print(f"Обработано стратегиями: {', '.join(summary['strategies_applied'])}")
        print(f"Исходный размер: {X.shape}")
        print(f"Обработанный размер: {X_processed.shape}")
        print(f"Удалено строк: {summary['removed_rows']} ({result['removal_percent']:.1f}%)")
        print(f"Создано бинарных признаков: {len(summary['binary_features_created'])}")
        print(f"Целевая переменная: {y_processed.shape}")
        
        if summary['binary_features_created']:
            print(f"Бинарные признаки: {', '.join(summary['binary_features_created'])}")
        
        # Детали по выбросам
        print("\nДетали по выбросам:")
        for feature, outliers in summary['outliers_by_feature'].items():
            for outlier_info in outliers:
                print(f"  - {feature}: {outlier_info['count']} (порог: {outlier_info['threshold']})")
                
    except Exception as e:
        print(f"Ошибка при обработке стратегией {strategies}: {e}")
        results.append({
            'strategies': strategies,
            'error': str(e),
            'X_processed': X.shape[0],
            'rows_removed': 0
        })


СТРАТЕГИЯ 1: ['abnormal']
Обработано стратегиями: abnormal: 11 rows
Исходный размер: (781, 10)
Обработанный размер: (770, 10)
Удалено строк: 11 (1.4%)
Создано бинарных признаков: 0
Целевая переменная: (770,)

Детали по выбросам:
  - Sp/C_pct: 8 (порог: 0.098)
  - Fine Aggregate: 3 (порог: 970)

СТРАТЕГИЯ 2: ['combine']
Обработано стратегиями: combine: 11 rows
Исходный размер: (781, 10)
Обработанный размер: (770, 12)
Удалено строк: 11 (1.4%)
Создано бинарных признаков: 2
Целевая переменная: (770,)
Бинарные признаки: High_SP, High_FA

Детали по выбросам:
  - High_SP: 380 (порог: 0.025)
  - High_FA: 293 (порог: 800)

СТРАТЕГИЯ 3: ['gost_binar']
Обработано стратегиями: 
Исходный размер: (781, 10)
Обработанный размер: (781, 18)
Удалено строк: 0 (0.0%)
Создано бинарных признаков: 7
Целевая переменная: (781,)
Бинарные признаки: Outlier_cement, Outlier_coarse_aggregate, Outlier_fine_aggregate, Outlier_fly_ash, Outlier_sp_c_pct, Outlier_w_c, Outlier_water

Детали по выбросам:
  - cement: 202 (

In [21]:
# Сводная таблица результатов
print(f"\n{'='*90}")
print("СВОДНАЯ ТАБЛИЦА РЕЗУЛЬТАТОВ")
print(f"{'='*90}")

summary_df = pd.DataFrame([
    {
        'Стратегия': ', '.join(r['strategies']),
        'Исходные строки': r['X_original'],
        'Исходные признаки': X.shape[1],
        # 'Обработанные строки': r['X_processed'],
        # 'Обработанные признаки': r['X_processed_data'].shape[1],
        'Размер данных': f"{r['X_processed']}×{r['X_processed_data'].shape[1]}",
        'Удалено строк': r.get('rows_removed', 0),
        'Удалено %': f"{r.get('removal_percent', 0):.1f}%",
        'Бинарные признаки': r.get('binary_features', 0),
        'Статус': 'Успех' if 'error' not in r else f"Ошибка: {r['error']}"
    }
    for r in results if 'X_processed_data' in r
])

print(summary_df.to_string(index=False))


СВОДНАЯ ТАБЛИЦА РЕЗУЛЬТАТОВ
          Стратегия  Исходные строки  Исходные признаки Размер данных  Удалено строк Удалено %  Бинарные признаки Статус
           abnormal              781                 10        770×10             11      1.4%                  0  Успех
            combine              781                 10        770×12             11      1.4%                  2  Успех
         gost_binar              781                 10        781×18              0      0.0%                  7  Успех
        gost_remove              781                 10          6×10            775     99.2%                  0  Успех
         iqr_remove              781                 10        691×10             90     11.5%                  0  Успех
gost_binar, combine              781                 10        770×20             11      1.4%                  9  Успех
